## Wine Clustering

**Proposta:** Aplicar algoritmos de clusterização para identificar os diferentes tipos de vinho presentes no dataset a partir de sua composição química.

**Link do Kaggle:** https://www.kaggle.com/datasets/harrywang/wine-dataset-for-clustering

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import warnings
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN
from sklearn.cluster import MeanShift
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score



In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/CursoML/WineClustering/wine-clustering.csv',
                    sep=',', encoding='iso-8859-1')

### Exploração dos Dados

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
df.isnull().sum()

In [ ]:
df.describe()

In [ ]:
boxplot = px.box(df, y="Proline")
boxplot.show()

Têm alguns outliers: Malic_Acid, Ash, Ash_Alcanity, Magnesium, Proanthocyanins, Color_Intensity, Hue.

### Pré-processamento

**Escalonamento**

In [ ]:
escala = StandardScaler()
df_esc = escala.fit_transform(df)

In [ ]:
df

In [ ]:
df_esc

### K-Means com PCA

In [ ]:
pca = PCA(n_components = 2)

In [ ]:
df_pca = pca.fit_transform(df_esc)

In [ ]:
pca.explained_variance_ratio_

Elbow Method

In [ ]:
wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', random_state = 5, max_iter = 300)
    kmeans.fit(df_pca)
    wcss.append(kmeans.inertia_)

In [ ]:
plt.figure(figsize=(10,5))
sns.lineplot(x=range(1, 11), y=wcss,marker='o',color='red')
plt.title('The Elbow Method')
plt.xlabel('Número de clusters')
plt.ylabel('WCSS');

In [ ]:
kmeans = KMeans(n_clusters = 3, init = 'k-means++', random_state = 5, max_iter = 300)


In [ ]:
kmeans2 = kmeans.fit(df_pca)

In [ ]:
centroides = kmeans2.cluster_centers_

In [ ]:
classificacao1 = kmeans2.labels_
classificacao1

In [ ]:
graf1 = px.scatter(x = df_pca[:,0], y = df_pca[:,1], color=classificacao1)
graf2 = px.scatter(x = centroides[:,0], y = centroides[:,1], size = [15, 15, 15])
graf3 = go.Figure(data = graf1.data + graf2.data)
graf3.update_layout(width=800,height=500,title_text='Agrupamento K-Means com PCA')
graf3.update_xaxes(title = 'Componente 1')
graf3.update_yaxes(title = 'Componente 2')
graf3.show()

In [ ]:
agrupamento1 = pd.DataFrame(classificacao1, columns = ['Grupo'])
agrupamento1

In [ ]:
df2 = pd.concat([df,agrupamento1],axis=1)
df2

In [ ]:
grupo0 = df2.loc[df2['Grupo'] == 0]
grupo1 = df2.loc[df2['Grupo'] == 1]
grupo2 = df2.loc[df2['Grupo'] == 2]

In [ ]:
grupo0.shape

In [ ]:
grupo1.shape

In [ ]:
grupo2.shape

In [ ]:
sil1 = silhouette_score(df_pca, classificacao1)
dbi1 = davies_bouldin_score(df_pca, classificacao1)
chi1 = calinski_harabasz_score(df_pca, classificacao1)

print("Silhouette:", sil1, "DBI:", dbi1, "CHI:", chi1)

In [ ]:
resultados = []
resultados.append(["KMeans", "Sim", 3, sil1, dbi1, chi1])

### K-Means com todos os atributos

In [ ]:
from sklearn.cluster import KMeans
wcss = []
for i in range(1,15):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', random_state = 5, max_iter = 300)
    kmeans.fit(df_esc)
    wcss.append(kmeans.inertia_)

In [ ]:
plt.figure(figsize=(10,5))
sns.lineplot(x=range(1, 15), y=wcss,marker='o',color='red')
plt.title('The Elbow Method')
plt.xlabel('Número de clusters')
plt.ylabel('WCSS');

In [ ]:
kmeans = KMeans(n_clusters = 3, init = 'k-means++', random_state = 5, max_iter = 300)

In [ ]:
kmeans3 = kmeans.fit(df_esc)

In [ ]:
centroides = kmeans3.cluster_centers_
centroides

In [ ]:
classificacao2 = kmeans3.labels_
classificacao2

In [ ]:
agrupamento2 = pd.DataFrame(classificacao2, columns = ['Grupo'])

In [ ]:
df3 = pd.concat([df,agrupamento2],axis=1)
df3

In [ ]:
grupo0 = df3.loc[df3.Grupo == 0]
grupo1 = df3.loc[df3.Grupo == 1]
grupo2 = df3.loc[df3.Grupo == 2]

In [ ]:
grupo0.shape

In [ ]:
grupo1.shape

In [ ]:
grupo2.shape

In [ ]:
sil2 = silhouette_score(df_esc, classificacao2)
dbi2 = davies_bouldin_score(df_esc, classificacao2)
chi2 = calinski_harabasz_score(df_esc, classificacao2)

print("Silhouette:", sil2, "DBI:", dbi2, "CHI:", chi2)

In [ ]:
resultados.append(["KMeans", "Não", 3, sil2, dbi2, chi2])

### Agrupamento Hierárquico com PCA

In [ ]:
dendrograma = dendrogram(linkage(df_pca, method = 'complete'))

In [ ]:
hier = AgglomerativeClustering(n_clusters=3, metric='euclidean', linkage = 'complete')
classificacao3 = hier.fit_predict(df_pca)

In [ ]:
classificacao3

In [ ]:
graf = px.scatter(x = df_pca[:,0], y = df_pca[:,1], color=classificacao3)
graf.update_layout(width=800,height=500,title_text='Agrupamento Hierárquico')
graf.show()

In [ ]:
agrupamento3 = pd.DataFrame(classificacao3, columns = ['Grupo'])

In [ ]:
df4 = pd.concat([df,agrupamento3],axis=1)
df4

In [ ]:
grupo0 = df4.loc[df4.Grupo == 0]
grupo1 = df4.loc[df4.Grupo == 1]
grupo2 = df4.loc[df4.Grupo == 2]

In [ ]:
grupo0.shape

In [ ]:
grupo1.shape

In [ ]:
grupo2.shape

In [ ]:
sil3 = silhouette_score(df_pca, classificacao3)
dbi3 = davies_bouldin_score(df_pca, classificacao3)
chi3 = calinski_harabasz_score(df_pca, classificacao3)

print("Silhouette:", sil3, "DBI:", dbi3, "CHI:", chi3)

In [ ]:
resultados.append(["Hierárquico", "Sim", 3, sil3, dbi3, chi3])

### Agrupamento hierárquico com todos os atributos

In [ ]:
dendrograma = dendrogram(linkage(df_esc, method = 'ward'))

In [ ]:
hier = AgglomerativeClustering(n_clusters=3, metric='euclidean', linkage = 'ward')
classificacao4 = hier.fit_predict(df_esc)

In [ ]:
classificacao4

In [ ]:
agrupamento4 = pd.DataFrame(classificacao4, columns = ['Grupo'])

In [ ]:
df5 = pd.concat([df,agrupamento4],axis=1)

In [ ]:
grupo0 = df5.loc[df5.Grupo == 0]
grupo1 = df5.loc[df5.Grupo == 1]
grupo2 = df5.loc[df5.Grupo == 2]

In [ ]:
grupo0.shape

In [ ]:
grupo1.shape

In [ ]:
grupo2.shape

In [ ]:
sil4 = silhouette_score(df_esc, classificacao4)
dbi4 = davies_bouldin_score(df_esc, classificacao4)
chi4 = calinski_harabasz_score(df_esc, classificacao4)

print("Silhouette:", sil4, "DBI:", dbi4, "CHI:", chi4)

In [ ]:
resultados.append(["Hierárquico", "Não", 3, sil4, dbi4, chi4])

### DBSCAN com PCA

In [ ]:
dbscan = DBSCAN(eps = 0.53, min_samples=4)
dbscan.fit(df_pca)

In [ ]:
classificacao5 = dbscan.labels_
classificacao5

In [ ]:
graf = px.scatter(x = df_pca[:,0], y = df_pca[:,1], color=classificacao5)
graf.update_layout(width=800,height=500,title_text='Agrupamento DBSCAN')
graf.show()

In [ ]:
agrupamento5 = pd.DataFrame(classificacao5, columns = ['Grupo'])
agrupamento5

In [ ]:
df6 = pd.concat([df, agrupamento5],axis=1)

In [ ]:
grupo0 = df6.loc[df6.Grupo == 0]
grupo1 = df6.loc[df6.Grupo == 1]
grupo2 = df6.loc[df6.Grupo == 2]

In [ ]:
grupo0.shape

In [ ]:
grupo1.shape

In [ ]:
grupo2.shape

In [ ]:
sil5 = silhouette_score(df_pca, classificacao5)
dbi5 = davies_bouldin_score(df_pca, classificacao5)
chi5 = calinski_harabasz_score(df_pca, classificacao5)

print("Silhouette:", sil5, "DBI:", dbi5, "CHI:", chi5)

In [ ]:
resultados.append(["DBSCAN", "Sim", 3, sil5, dbi5, chi5])

### DBSCAN com todos os atributos

In [ ]:
from sklearn.neighbors import NearestNeighbors
import numpy as np
import matplotlib.pyplot as plt

neighbors = NearestNeighbors(n_neighbors=3)
neighbors_fit = neighbors.fit(df_esc)
distances, indices = neighbors_fit.kneighbors(df_esc)

distances = np.sort(distances[:,2])
plt.plot(distances)
plt.show()

In [ ]:
dbscan = DBSCAN(eps = 2.16, min_samples=3)
dbscan.fit(df_esc)

In [ ]:
classificacao6 = dbscan.labels_
classificacao6

In [ ]:
agrupamento6 = pd.DataFrame(classificacao6, columns = ['Grupo'])


In [ ]:
df7 = pd.concat([df,agrupamento6],axis=1)

In [ ]:
grupo0 = df7.loc[df7.Grupo == 0]
grupo1 = df7.loc[df7.Grupo == 1]
grupo2 = df7.loc[df7.Grupo == 2]

In [ ]:
grupo0.shape

In [ ]:
grupo1.shape

In [ ]:
grupo2.shape

In [ ]:
sil6 = silhouette_score(df_esc, classificacao6)
dbi6 = davies_bouldin_score(df_esc, classificacao6)
chi6 = calinski_harabasz_score(df_esc, classificacao6)

print("Silhouette:", sil6, "DBI:", dbi6, "CHI:", chi6)

In [ ]:
resultados.append(["DBSCAN", "Não", 3, sil6, dbi6, chi6])

### MeanShift com PCA

In [ ]:
mean = MeanShift(bandwidth=1.3, cluster_all=True)

In [ ]:
mean.fit(df_pca)

In [ ]:
classificacao7 = mean.labels_
classificacao7

In [ ]:
agrupamento7 = pd.DataFrame(classificacao7, columns = ['Grupo'])

In [ ]:
df8 = pd.concat([df,agrupamento7],axis=1)

In [ ]:
grupo0 = df8.loc[df8.Grupo == 0]
grupo1 = df8.loc[df8.Grupo == 1]
grupo2 = df8.loc[df8.Grupo == 2]
grupo3 = df8.loc[df8.Grupo == 3]

In [ ]:
grupo0.shape

In [ ]:
grupo1.shape

In [ ]:
grupo2.shape

In [ ]:
grupo3.shape

In [ ]:
sil7 = silhouette_score(df_pca, classificacao7)
dbi7 = davies_bouldin_score(df_pca, classificacao7)
chi7 = calinski_harabasz_score(df_pca, classificacao7)

print("Silhouette:", sil7, "DBI:", dbi7, "CHI:", chi7)

In [ ]:
resultados.append(["MeanShift", "Sim", 4, sil7, dbi7, chi7])

### MeanShift com todos os atributos

In [ ]:
mean = MeanShift(bandwidth=3.6, cluster_all=True)

In [ ]:
mean.fit(df_esc)

In [ ]:
classificacao8 = mean.labels_
classificacao8

In [ ]:
agrupamento8 = pd.DataFrame(classificacao8, columns = ['Grupo'])

In [ ]:
df9 = pd.concat([df,agrupamento8],axis=1)

In [ ]:
grupo0 = df9.loc[df9.Grupo == 0]
grupo1 = df9.loc[df9.Grupo == 1]
grupo2 = df9.loc[df9.Grupo == 2]
grupo3 = df9.loc[df9.Grupo == 3]
grupo4 = df9.loc[df9.Grupo == 4]
grupo5 = df9.loc[df9.Grupo == 5]

In [ ]:
grupo0.shape

In [ ]:
grupo1.shape

In [ ]:
grupo2.shape

In [ ]:
grupo3.shape

In [ ]:
grupo4.shape

In [ ]:
grupo5.shape

In [ ]:
sil8 = silhouette_score(df_esc, classificacao8)
dbi8 = davies_bouldin_score(df_esc, classificacao8)
chi8 = calinski_harabasz_score(df_esc, classificacao8)

print("Silhouette:", sil8, "DBI:", dbi8, "CHI:", chi8)

In [ ]:
resultados.append(["MeanShift", "Não", 6, sil8, dbi8, chi8])

### Tabela comparativa

In [ ]:
tabela = pd.DataFrame(resultados, columns=["Algoritmo", "PCA", "#Clusters", "Silhouette", "DBI", "CHI"])
tabela